# Health Data Science for Diabetes Tutorial
- Tutors: Minhong Wang, Honghan Wu, Debbie Wake
- University of Edinburgh
- March 2021

## Summary
This tutorial illustrates the streamlined pipeline of applying health data science on routine data for research.

## Table of Content
1. Data model mapping and preprocessing
    - Common Data Model
    - Standadised Terminology
2. Phenotype Computation
    - Type 2 Diabetes by Diagnosis
    - Type 2 Diabetes by Medication
    - Drug episodes
3. Exploratory Data Analysis
    - dataframe basics
    - exploration and visualisation
    - outlier detection and removal
    - imputation
4. Risk Prediction
    - Cox model for CVD risk prediction

## 1. Data model mapping and preprocessing
Why do we need common data model?

![Why Common Data Model](https://www.ohdsi.org/wp-content/uploads/2014/07/Why-CDM.png "schematic diagram of CDM")

## Our data tables
- `Demography.csv` - demography table about patient basic information such as age, gender etc. 
- `Biochemistry.csv` - laboratory test results including test code, result values, units etc.
- `Prescribing.csv` - drug prescriptions including drug names, drug codes, dosage, prescription time etc.
- `HospitalAdmissions.csv` -  hospital admissions including admission date, discharge date, diagnosis etc.

### R packages
An R package is an extension of R containing data sets and specific functions to solve specific questions. In addition to many base packages, there are thousands of other R packages available for download and installation. For first time using these packages, we need to install these packages and also load the packages for using the functions in the packages.

All the packages we use for this tutorial are already installed. So we only need to load the packages here.

In [15]:
#install.packages("tidyverse")
#install.packages("dplyr")
#install.packages("lubridate")
#install.packages("chron")
library(tidyverse)
library(dplyr)
library(lubridate) # date convert
library(chron) # datetime

### Working directory
The working directory is a folder where R reads and saves files. Here, we set our working directory to the folder called `data`.

In [17]:
# set working directory
setwd("./data")

ERROR: Error in setwd("./data"): cannot change working directory


## 1.1 Map patient demographic information to CDM Person table
Suppose our own demographic table is called `Demography` and we have extracted the table as a `CSV`file. 

In [18]:
# read the csv file into R as a dataframe
demography <- read.csv("Demography.csv",header = TRUE, sep = ",",stringsAsFactors = FALSE)
head(demography)

,chi,dtCreated,current_record,notes,chi_num_of_curr_record,chi_status,century,surname,forename,sex,⋯,previous_postcode,date_address_changed,adr,current_gp_accept_date,previous_gp,previous_gp_accept_date,date_into_practice,date_of_birth,patient_triage_score,hic_dataLoadRunID
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
1,2807076911,17/02/2008 05:20:23,False,Random record,NA,L,20,Bell,MATILDA,F,⋯,,,NA,03/12/2010 23:12:02,Z972,14/08/2012 12:47:50,16/01/2008 16:48:38,28/07/2007 08:21:03,8.70,11
2,2108353910,14/04/1961 15:57:17,True,Random record,NA,L,19,Griffith,ELLA,F,⋯,,,NA,09/05/1984 10:34:23,,,04/08/1947 02:40:41,21/08/1935 09:44:39,0.61,7
3,1206409032,14/09/2003 16:21:15,True,Random record,NA,,19,Murphy,JASMINE,F,⋯,,,NA,19/04/2010 02:54:52,,,21/03/1955 07:37:45,12/06/1940 14:46:45,9.00,11
4,103806118,31/01/1982 05:49:30,False,Random record,NA,C,19,,MARYAM Ellis,F,⋯,,,NA,18/12/1998 12:04:36,,,09/05/1980 20:29:09,01/03/1980 03:08:43,0.88,9
5,1503028811,30/08/2017 20:00:49,False,Random record,NA,R,20,Bennett,SOFIA,F,⋯,,,NA,30/11/2013 13:44:07,,,02/11/2007 11:53:23,15/03/2002 21:08:33,0.10,12
6,1001408588,24/10/2016 00:21:03,True,Random record,NA,,19,Richards,TYLER,M,⋯,DD9 7JB,22/04/1945 22:52:25,NA,01/08/2012 05:41:19,P528,01/03/2019 13:01:19,13/01/1959 11:39:13,10/01/1940 15:29:35,7.40,12


In [19]:
# let's look at the column names and summary data
str(demography)

'data.frame':	10000 obs. of  39 variables:
 $ chi                    : num  2.81e+09 2.11e+09 1.21e+09 1.04e+08 1.50e+09 ...
 $ dtCreated              : chr  "17/02/2008 05:20:23" "14/04/1961 15:57:17" "14/09/2003 16:21:15" "31/01/1982 05:49:30" ...
 $ current_record         : chr  "False" "True" "True" "False" ...
 $ notes                  : chr  "Random record" "Random record" "Random record" "Random record" ...
 $ chi_num_of_curr_record : num  NA NA NA NA NA NA NA NA NA NA ...
 $ chi_status             : chr  "L" "L" "" "C" ...
 $ century                : int  20 19 19 19 20 19 19 20 19 19 ...
 $ surname                : chr  "Bell" "Griffith" "Murphy" "" ...
 $ forename               : chr  "MATILDA" "ELLA" "JASMINE" "MARYAM Ellis" ...
 $ sex                    : chr  "F" "F" "F" "F" ...
 $ current_address_L1     : chr  "41 Brown Street" "12 Dudhope Terrace" "20 Viscount's Pend" "Ballindean Place" ...
 $ current_address_L2     : chr  "Coldside" "Montrose and District" "West End" "E

---
### What is the equivilant in CDM?

The `Person` table and other related tables

![CDM Person related tables](https://www.ohdsi.org/web/wiki/lib/exe/fetch.php?w=800&tok=82724f&media=documentation:cdm:standard_health_system_data_tables.png "CDM Person related tables")

### NB: Before anything, we need a `Concept` table in CDM

In [20]:
# generate an empty concept table
concept <- data.frame(concept_id = numeric(),
                      concept_name = character(),
                      domain_id = character(),
                      vocabulary_id = character(),
                      concept_class_id = character(),
                      concept_code = character(),
                      valid_start_date = as.Date(character()),
                      valid_end_date = as.Date(character()),
                      stringsAsFactors = FALSE)

## Do necessary data column preprocessing 
- format the date of birth 
- generate three date/time columns for date of birth information, that are required by CDM `Person` table

In [21]:
# convert date of birth
demography$date_of_birth <- dmy_hms(demography$date_of_birth)

demography <- demography %>% dplyr::mutate(year_of_birth = lubridate::year(date_of_birth),
                                           month_of_birth = lubridate::month(date_of_birth),
                                           day_of_birth = lubridate::day(date_of_birth))

nrow(as.data.frame(unique(demography$chi)))

[1] 1000

In [22]:
# select the data of each patient based on the newest dtCreated
demography$dtCreated <- dmy_hms(demography$dtCreated)

demography <- demography %>% dplyr::group_by(chi) %>% dplyr::filter(dtCreated == max(dtCreated))

## Generate `person` dataframe from  `demography`
- need to populate `concept` entries along the way

In [26]:
person <- demography %>% select(chi,sex,year_of_birth)
#str(person)

# column: gender concept id --> 1 for male, 2 for female
person$gender_concept_id <- ifelse(person$sex=="M",1,2)
concept[nrow(concept)+1,] = list(1,"male","gender","na","na","na","1970/01/01","2099/12/31")
concept[nrow(concept)+1,] = list(2,"female","gender","na","na","na","1970/01/01","2099/12/31")

# column: race concept id (not included in badMedicine data) --> 3 for Chinese
person$race_concept_id <- 3
concept[nrow(concept)+1,] = list(3,"chinese","race","na","na","na","1970/01/01","2099/12/31")

# column: ethnicity concept id (not included in bad Medicine data) --> 4 for non hispanic
person$ethnicity_concept_id <- 4
concept[nrow(concept)+1,] = list(4,"non hispanic","ethnicity","na","na","na","1970/01/01","2099/12/31")

# column: gender source concept id
person$gender_source_concept_id <- 0
concept[nrow(concept)+1,] = list(0, "no resource", "na", "na", "na","na","1970/01/01","2099/12/31")

# column: race_source_concept_id
person$race_source_concept_id <- 0

# column: ethnicity source concept id
person$ethnicity_source_concept_id <- 0

# reoder column for import
person <- person %>% select(chi,
                            gender_concept_id,
                            year_of_birth,
                            race_concept_id,
                            ethnicity_concept_id,
                            gender_source_concept_id,
                            race_source_concept_id,
                            ethnicity_source_concept_id
)

In [25]:
head(person)

chi,gender_concept_id,year_of_birth,race_concept_id,ethnicity_concept_id,gender_source_concept_id,race_source_concept_id,ethnicity_source_concept_id
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1001408588,1,1940,3,4,0,0,0
2709858218,2,1985,3,4,0,0,0
1207378126,1,1937,3,4,0,0,0
1711545825,1,1954,3,4,0,0,0
409056284,1,2005,3,4,0,0,0
2008837417,2,1983,3,4,0,0,0


## save the `person` dataframe as a csv file, which will be loaded the CDM database  

In [27]:
### output
write.csv(person, "omop_person.csv", row.names = FALSE)

# Task 1. Map `Prescribing.csv` into CDM tables

In [30]:
###################
### prescribing ###
###################
prescribing <- read.csv("Prescribing.csv",header = TRUE, sep = ",",stringsAsFactors = FALSE)

#################
# drug exposure #
#################

# required: drug_exposure_id, person_id, drug_concept_id,  drug_exposure_start_datetime, drug_exposure_end_datetime, drug_type_concept_id, drug_source_concept_id
# further work on quantity#############################################

drug_exposure <- prescribing %>% select(chi,
                                        BnfCode,
                                        PrescribedDate)


drug_exposure$drug_type_concept_id <- 0

drug_exposure$drug_source_concept_id <- 0

drug_exposure$route_concept_id <- 0

# for drug_concept_id
# bnf code --> concept id
# self defined here
bnf_list <- data.frame(unique(drug_exposure$BnfCode)) %>% drop_na()
bnf_list$drug_concept_id <- seq(10001,nrow(bnf_list)+10000,by=1)

drug_exposure <- left_join(drug_exposure,bnf_list,by = c("BnfCode" = "unique.drug_exposure.BnfCode."))

# add to concept table
concept_tmp <- data.frame(bnf_list$drug_concept_id) %>% rename(concept_id = bnf_list.drug_concept_id)
concept_tmp$concept_name <- bnf_list$unique.drug_exposure.BnfCode.
concept_tmp$domain_id <- "drug"
concept_tmp$vocabulary_id <- "bnf"
concept_tmp$concept_class_id <- "na"
concept_tmp$concept_code <- "na"
concept_tmp$valid_start_date <- "1970/01/01"
concept_tmp$valid_end_date <- "2099/12/31"

concept <- rbind(concept,concept_tmp)

# assume prescribed date is start date
drug_exposure$drug_exposure_start_datetime <- as.POSIXlt(drug_exposure$PrescribedDate, format = "%d/%m/%Y %H:%M:%OS")

drug_exposure$drug_exposure_end_datetime <- as.POSIXlt("2099/12/31 00:00:00")

# id
drug_exposure_id <- rownames(drug_exposure)
drug_exposure <- cbind(drug_exposure_id,drug_exposure)

# reorder columns and output csv
drug_exposure <- drug_exposure %>% select(drug_exposure_id,
                                          chi,
                                          drug_concept_id,
                                          drug_exposure_start_datetime,
                                          drug_exposure_end_datetime,
                                          drug_type_concept_id,
                                          route_concept_id,
                                          drug_source_concept_id)

head(drug_exposure)
write.csv(drug_exposure, "omop_drug_exposure.csv", row.names = FALSE)


,drug_exposure_id,chi,drug_concept_id,drug_exposure_start_datetime,drug_exposure_end_datetime,drug_type_concept_id,route_concept_id,drug_source_concept_id
,<chr>,<dbl>,<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>
1,1,2306555010,10001,1997-01-06 18:47:13,2099-12-31,0,0,0
2,2,2105917407,10002,1997-11-27 00:00:14,2099-12-31,0,0,0
3,3,3004926220,10003,2013-05-21 09:16:19,2099-12-31,0,0,0
4,4,309935235,10004,2000-10-04 17:31:09,2099-12-31,0,0,0
5,5,2405874216,10001,2017-07-18 13:36:19,2099-12-31,0,0,0
6,6,1502223358,10005,1937-03-19 05:34:38,2099-12-31,0,0,0


# Task 2. Map `HospitalAdmissions.csv` into CDM tables

### Task 2.1 `visit` tables

In [33]:
###############################
### hospital administration ###
###############################
hospital_ad <- read.csv("HospitalAdmissions.csv",header = TRUE, sep = ",",stringsAsFactors = FALSE)

####################
# visit occurrence #
####################
# required: visit_occurrence_id, person_id, visit_concept_id, visit_start_datetime, visit_end_datetime, visit_type_concept_id, visit_source_concept_id, admitted_from_concept_id, discharge_to_concept_id
# we also have: visit_start_date, visit_end_date

# get data from hospitcal_ad
visit_occurrence <- hospital_ad %>% select(chi,AdmissionDate,DischargeDate)


# visit concept id
# visit concept:https://athena.ohdsi.org/search-terms/terms?domain=Visit&standardConcept=Standard&page=7&pageSize=15&query=
# visit concept: e.g. inpatient visit, outpatient visit, emergency, ambulance etc
# cause our synthetic data is mimicksing smr01, assume they are all inpatient visit
visit_occurrence$visit_concept_id <- 5
concept[nrow(concept)+1,] = list(5,"inpatient visit","visit type","na","na","na","1970/01/01","2099/12/31")

# visit_start_datetime and visit_end_datetime
visit_occurrence$visit_start_datetime <- as.POSIXlt("1900/01/01 00:00:00")
visit_occurrence$visit_end_datetime <- as.POSIXlt("2099/12/31 00:00:00")


# visit type concept id
# provenance of the visit record/ where the record comes from
# https://athena.ohdsi.org/search-terms/terms?domain=Type+Concept&standardConcept=Standard&page=1&pageSize=15&query=
visit_occurrence$visit_type_concept_id <- 0

#visit_occurrence$visit_source_concept_id
visit_occurrence$visit_source_concept_id <- 0
#visit_occurrence$admitted_from_concept_id
visit_occurrence$admitted_from_concept_id <- 0
#visit_occurrence$discharge_to_concept_id
visit_occurrence$discharge_to_concept_id <- 0

# visit_start_date and visit_end_date
visit_occurrence$visit_start_date <- as.Date(dmy_hms(visit_occurrence$AdmissionDate))
visit_occurrence$visit_end_date <- as.Date(dmy_hms(visit_occurrence$DischargeDate))

# visit_occurrence_id
visit_occurrence_id <- rownames(visit_occurrence)
visit_occurrence <- cbind(visit_occurrence_id,visit_occurrence)


visit_occurrence <- visit_occurrence %>% select(visit_occurrence_id,
                                                chi,
                                                visit_concept_id,
                                                visit_start_date,
                                                visit_start_datetime,
                                                visit_end_date,
                                                visit_end_datetime,
                                                visit_type_concept_id,
                                                visit_source_concept_id,
                                                admitted_from_concept_id,
                                                discharge_to_concept_id)
head(visit_occurrence)
### output
write.csv(visit_occurrence, "omop_visit_occurence.csv", row.names = FALSE)

,visit_occurrence_id,chi,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,visit_source_concept_id,admitted_from_concept_id,discharge_to_concept_id
,<chr>,<dbl>,<dbl>,<date>,<dttm>,<date>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1202666427,5,2017-08-10,1900-01-01,2017-08-19,2099-12-31,0,0,0,0
2,2,202826676,5,1998-12-21,1900-01-01,1998-12-22,2099-12-31,0,0,0,0
3,3,2308797724,5,2004-04-09,1900-01-01,2004-04-19,2099-12-31,0,0,0,0
4,4,908238013,5,2007-01-01,1900-01-01,2007-01-01,2099-12-31,0,0,0,0
5,5,703751920,5,1997-10-21,1900-01-01,1997-10-25,2099-12-31,0,0,0,0
6,6,912538317,5,2016-02-23,1900-01-01,2016-03-01,2099-12-31,0,0,0,0


### Task 2.2 `Condition` tables

In [38]:
########################
# condition occurrence #
########################
# e.g. diagnosis, sign, symptom observed
# required: condition_occurrence_id, person_id, condition_concept_id, condition_start_date, condition_type_concept_id, condition_status_concept_id, condition_source_concept_id, 
condition_occurrence <- hospital_ad %>% select(chi,
                                               MainCondition,
                                               OtherCondition1,
                                               OtherCondition2,
                                               OtherCondition3)


condition_occurrence[condition_occurrence==""] <- NA
condition_occurrence[condition_occurrence=="null"] <- NA
condition_occurrence[condition_occurrence=="NULL"] <- NA
condition_occurrence[condition_occurrence=="nul"] <- NA
condition_occurrence[condition_occurrence=="Nul"] <- NA

condition1 <- condition_occurrence %>% select(chi,OtherCondition1) %>% drop_na(OtherCondition1)
condition2 <- condition_occurrence %>% select(chi,OtherCondition2) %>% drop_na(OtherCondition2)
condition3 <- condition_occurrence %>% select(chi,OtherCondition3) %>% drop_na(OtherCondition3)

condition_occurrence <- condition_occurrence %>% select(chi,MainCondition)

condition_occurrence <- rbind(condition_occurrence,
                              setNames(condition1,names(condition_occurrence)),
                              setNames(condition2,names(condition_occurrence)),
                              setNames(condition3,names(condition_occurrence)))



# using the unique conditions appeared in this table to generate condition concept id
condition_list <- data.frame(unique(condition_occurrence$MainCondition)) %>% drop_na()
condition_list$condition_concept_id <- seq(1001,nrow(condition_list)+1000,by=1)

condition_occurrence <- left_join(condition_occurrence,condition_list,by = c("MainCondition"="unique.condition_occurrence.MainCondition."))

# add concepts to concept table
concept_tmp <- data.frame(condition_list$condition_concept_id) %>% rename(concept_id = condition_list.condition_concept_id)
concept_tmp$concept_name <- condition_list$unique.condition_occurrence.MainCondition.
concept_tmp$domain_id <- "condition"
concept_tmp$vocabulary_id <- "icd10"
concept_tmp$concept_class_id <- "na"
concept_tmp$concept_code <- "na"
concept_tmp$valid_start_date <- "1970/01/01"
concept_tmp$valid_end_date <- "2099/12/31"

concept <- rbind(concept,concept_tmp)

# condition start date and end date (no resource)
condition_occurrence$condition_start_date <- as.Date("1900/01/01")
condition_occurrence$condition_end_date <- as.Date("2099/12/31")
condition_occurrence$condition_start_datetime <- as.POSIXlt("1900/01/01 00:00:00")

# condition type concept id (provenance of the condition records)
condition_occurrence$condition_type_concept_id <- 0

# condition status concept (e.g.admitting diagnosis, final diagnosis etc) (no information)
condition_occurrence$condition_status_concept_id <- 0

# condition source concept id
condition_occurrence$condition_source_concept_id <- 0

# id
condition_occurrence_id <- rownames(condition_occurrence)
condition_occurrence <- cbind(condition_occurrence_id,condition_occurrence)

# reorder columns and output
condition_occurrence <- condition_occurrence %>% select(condition_occurrence_id,
                                                        chi,
                                                        condition_concept_id,
                                                        condition_start_date,
                                                        condition_start_datetime,
                                                        condition_end_date,
                                                        condition_type_concept_id,
                                                        condition_status_concept_id,
                                                        condition_source_concept_id)


head(condition_occurrence)
write.csv(condition_occurrence, "omop_condition_occurrence.csv", row.names = FALSE)

,condition_occurrence_id,chi,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_type_concept_id,condition_status_concept_id,condition_source_concept_id
,<chr>,<dbl>,<dbl>,<date>,<dttm>,<date>,<dbl>,<dbl>,<dbl>
1,1,1202666427,1001,1900-01-01,1900-01-01,2099-12-31,0,0,0
2,2,202826676,1002,1900-01-01,1900-01-01,2099-12-31,0,0,0
3,3,2308797724,1003,1900-01-01,1900-01-01,2099-12-31,0,0,0
4,4,908238013,1004,1900-01-01,1900-01-01,2099-12-31,0,0,0
5,5,703751920,1005,1900-01-01,1900-01-01,2099-12-31,0,0,0
6,6,912538317,1006,1900-01-01,1900-01-01,2099-12-31,0,0,0


# Task 3. Map `Biochemistry.csv` into CDM tables

In [40]:
####################
### biochemistry ###
####################
biochemistry <- read.csv("Biochemistry.csv",header = TRUE, sep = ",",stringsAsFactors = FALSE)

###############
# measurement #
###############

# required: measurement_id, person_id, measurement_concept_id, measurement_datetime, measurement_type_concept_id, measurement_source_concept_id
# will also look at: value_as_number, value_as_concept_id, unit_concept_id#######################

measurement <- biochemistry %>% select(chi,
                                       SampleDate,
                                       TestCode,
                                       Result,
                                       QuantityUnit)



measurement$measurement_type_concept_id <- 0

measurement$measurement_source_concept_id <- 0

# test code --> concept id
# self defined here
testcode_list <- data.frame(unique(measurement$TestCode)) %>% drop_na()
testcode_list$measurement_concept_id <- seq(100001,nrow(testcode_list)+100000, by = 1)

measurement <- left_join(measurement,testcode_list,by = c("TestCode" = "unique.measurement.TestCode."))

# measurement_concept_id na replaced by 0
measurement$measurement_concept_id[is.na(measurement$measurement_concept_id)] <- 0

# add to concept table
concept_tmp <- data.frame(testcode_list$measurement_concept_id) %>% rename(concept_id = testcode_list.measurement_concept_id)
concept_tmp$concept_name <- testcode_list$unique.measurement.TestCode.
concept_tmp$domain_id <- "measurement"
concept_tmp$vocabulary_id <- "test code"
concept_tmp$concept_class_id <- "na"
concept_tmp$concept_code <- "na"
concept_tmp$valid_start_date <- "1970/01/01"
concept_tmp$valid_end_date <- "2099/12/31"

concept <- rbind(concept,concept_tmp)

# date
measurement$measurment_datetime <- as.POSIXlt(paste(measurement$SampleDate,"00:00:00"),format = "%d/%m/%Y %H:%M:%OS")

# result and quantityunit

measurement$value_as_number <- measurement$Result
measurement$value_as_number[is.na(measurement$value_as_number)] <- 0

measurement$unit_source_value <- measurement$QuantityUnit
######## unit need further work



# id
measurement_id <- rownames(measurement)
measurement <- cbind(measurement_id,measurement)

# reoder columns
measurement <- measurement %>% select(measurement_id,
                                      chi,
                                      measurement_concept_id,
                                      measurment_datetime,
                                      measurement_type_concept_id,
                                      value_as_number,
                                      measurement_source_concept_id,
                                      unit_source_value)


head(measurement)

### output
write.csv(measurement, "omop_measurement.csv", row.names = FALSE)


,measurement_id,chi,measurement_concept_id,measurment_datetime,measurement_type_concept_id,value_as_number,measurement_source_concept_id,unit_source_value
,<chr>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<chr>
1,1,2205275324,100001,2011-01-01 07:34:16,0,60.000000,0,mL/min
2,2,610397156,100002,1948-04-07 00:12:56,0,1.735227,0,g/L
3,3,2001699015,100003,2013-12-07 18:23:51,0,3.114884,0,mmol/L
4,4,408307572,100004,2012-01-13 12:50:55,0,4.212943,0,mmol/L
5,5,1108848748,100005,1999-06-26 11:13:39,0,5.099626,0,NULL
6,6,2410523124,100006,1967-12-10 06:05:47,0,135.898107,0,mg/L


## Finnally, do NOT forget to populate `concept` tables

In [41]:
###########
# concept #
###########

write.csv(concept,"omop_concept.csv",row.names = FALSE)
